In [1]:
import sqlite3


old_db_conn = sqlite3.connect('./backup/old.db')
new_db_conn = sqlite3.connect('./kwu-lecture-db-v2.db')


old_cursor = old_db_conn.cursor()
new_cursor = new_db_conn.cursor()


old_cursor.execute('''
SELECT lecCanTakeBunban, lecMajorRecogBunban, lecTakeOnly1Year, lecTakeOnly2Year, lecTakeOnly3Year, 
       lecTakeOnly4Year, lecTakeOnly5Year, lecForeignPeopleCanTake, lecCanTakeMultipleMajor, 
       lecTakeOnlyAthletics, lecTakeOnlyChambit, lecRequirementClass, lecLinkedMajorDifficulty, 
       year, semester, lecNumber
FROM LectureTable
''')


rows = old_cursor.fetchall()


for row in rows:
    canTakeBunBan = row[0]
    majorRecogBunBan = row[1]
    canTakeOnly1year = row[2]
    canTakeOnly2year = row[3]
    canTakeOnly3year = row[4]
    canTakeOnly4year = row[5]
    canTakeOnly5year = row[6]
    canTakeForeignPeople = row[7]
    canTakeMultipleMajor = row[8]
    canTakeOnlyAthlete = row[9]
    canTakeOnlyChambit = row[10]
    requirementClass = row[11]
    lecLinkedMajorDifficulty = row[12]
    year = row[13]
    semester = row[14]
    lecNumber = row[15]

    new_cursor.execute('''
    SELECT lectureID FROM LectureList
    WHERE year = ? AND semester = ? AND lecNumber = ?
    ''', (year, semester, lecNumber))

    lecture_id_row = new_cursor.fetchone()

    if lecture_id_row:
        lectureID = lecture_id_row[0]

        new_cursor.execute('''
        INSERT INTO LectureConditions (lectureID, canTakeBunBan, majorRecogBunBan, canTakeOnly1year, 
                                      canTakeOnly2year, canTakeOnly3year, canTakeOnly4year, 
                                      canTakeOnly5year, canTakeForeignPeople, canTakeMultipleMajor, 
                                      canTakeOnlyAthlete, canTakeOnlyChambit, requirementClass, 
                                      lecLinkedMajorDifficulty)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (lectureID, canTakeBunBan, majorRecogBunBan, canTakeOnly1year, canTakeOnly2year,
              canTakeOnly3year, canTakeOnly4year, canTakeOnly5year, canTakeForeignPeople,
              canTakeMultipleMajor, canTakeOnlyAthlete, canTakeOnlyChambit, requirementClass,
              lecLinkedMajorDifficulty))


new_db_conn.commit()


old_db_conn.close()
new_db_conn.close()

print("done")

done


In [4]:
import pandas as pd

def extract_unique_foreign_classes(file_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='cp949')
    
    # '분반_원본' 칼럼에서 '외국인'이 포함된 행 필터링
    foreign_classes = df[df['분반_원본'].str.contains('외국인', na=False)]
    
    # 필터링된 값들 중 유니크한 값 추출
    unique_foreign_classes = foreign_classes['분반_원본'].unique()
    
    return unique_foreign_classes

# 파일 경로 설정
file_path = 'yn_final_kwu-timetable5.csv'

# 함수 실행
result = extract_unique_foreign_classes(file_path)

# 결과 출력
print("유니크한 외국인 분반:")
for class_name in result:
    print("'"+class_name+"'")

유니크한 외국인 분반:
'외국인학생
수강불가'
'외국인만
수강가능'
'공대 합반
외국인만수강가능'
'전정대 합반
외국인만 수강가능'
'외국인만 수강가능'


In [16]:
import pandas as pd

if True:
    # CSV 파일 읽기
    df = pd.read_csv('yn_final_kwu-timetable5.csv', encoding='cp949')
    
    # '분반_원본' 열을 문자열로 변환하고 공백 및 줄바꿈 문자 제거
    df['분반_원본'] = df['분반_원본'].astype(str).str.strip().str.replace('\r\n', '')
    
    # 조건 리스트 정의 (공백 및 줄바꿈 문자 제거)
    class_name = ['외국인학생수강불가', '외국인만수강가능', '공대합반외국인만수강가능', '전정대합반외국인만 수강가능', '외국인만수강가능']
    class_name = [name.strip().replace('\r\n', '') for name in class_name]
    
    foreign_only = class_name[1:]
    korean_only = [class_name[0]]
    
    # 외국인 수강 가능 여부 확인 함수
    def is_foreign_allowed(value):
        if pd.isna(value) or value == 'nan':
            return 1
        if value in foreign_only:
            return 1
        elif value in korean_only:
            return 0
        return 1
    
    # 내국인 수강 가능 여부 확인 함수
    def is_korean_allowed(value):
        if pd.isna(value) or value == 'nan':
            return 1
        if value in korean_only:
            return 1
        elif value in foreign_only:
            return 0
        return 1
    
    # 새로운 칼럼 생성
    df['외국인_수강가능'] = df['분반_원본'].apply(is_foreign_allowed)
    df['내국인_수강가능'] = df['분반_원본'].apply(is_korean_allowed)
    
    # 처리된 데이터를 새 CSV 파일로 저장
    df.to_csv('processed_timetable.csv', index=False, encoding='cp949')
    
    # 결과 확인
    print(df[['분반_원본', '외국인_수강가능', '내국인_수강가능']].head(20))
    print("\n분반_원본 칼럼의 유니크한 값:")
    print(df['분반_원본'].unique())

                      분반_원본  외국인_수강가능  내국인_수강가능
0                 외국인학생수강불가         0         1
1                  외국인만수강가능         1         0
2   인공지능융합대학공통(C1~C9,J5~J8)         1         1
3                       nan         1         1
4                       nan         1         1
5                       nan         1         1
6                       nan         1         1
7                       nan         1         1
8                       nan         1         1
9                       nan         1         1
10                      nan         1         1
11                      nan         1         1
12                      nan         1         1
13                      nan         1         1
14                      nan         1         1
15                      nan         1         1
16     공과대학 공통(A1~A2,K1~K3)         1         1
17                      nan         1         1
18                      nan         1         1
19                      nan         1   

In [19]:
import pandas as pd

# 데이터프레임이 df라고 가정합니다
df = pd.read_csv('processed_timetable.csv', encoding='cp949')

# '학년' 열의 각 문자에 대해 새로운 열 생성
for grade in range(1, 5):
    df[f'{grade}학년'] = df['학년'].astype(str).str.contains(str(grade)).astype(int)

# 결과 확인
print(df)
    # 처리된 데이터를 새 CSV 파일로 저장
df.to_csv('processed_timetable.csv', index=False, encoding='cp949')

      Unnamed: 0            학정번호           과목명     테마      연계  이수   학점   시수  \
0              0  0000-1-8128-01         광운인되기  광운인되기     NaN  교필  1.0  1.0   
1              1  0000-1-8128-02         광운인되기  광운인되기     NaN  교선  1.0  1.0   
2              2  I000-1-3362-01          대학영어   대학영어     NaN  교필  3.0  3.0   
3              3  I000-1-3362-02          대학영어   대학영어     NaN  교필  3.0  3.0   
4              4  I000-1-3362-03          대학영어   대학영어     NaN  교필  3.0  3.0   
...          ...             ...           ...    ...     ...  ..  ...  ...   
1291        1647  0000-1-4849-01  배터리신기술과우리의생활    NaN  K-MOOC  교선  3.0  3.0   
1292        1648  0000-1-4850-01    수학적사고로보는세상    NaN  K-MOOC  교선  3.0  3.0   
1293        1649  0000-3-9747-01   안전및조직관리사례연구    NaN     군사학  무관  3.0  4.0   
1294        1650  0000-4-9748-01     조직리더십사례연구    NaN     군사학  무관  3.0  4.0   
1295        1651  0000-1-3675-01  외국인학생을위한기초수학    NaN    기초수학  교선  3.0  3.0   

     담당교수                     강의시간  ... 선수과목 추가인정분반

In [23]:
import csv
import sqlite3
import pandas as pd

# 데이터베이스 연결
conn = sqlite3.connect('kwu-lecture-recommendation-v4 copy.db')
cursor = conn.cursor()

# CSV 파일 읽기
df = pd.read_csv('processed_timetable.csv', encoding='cp949')

# 데이터베이스에 삽입하는 함수
def insert_lecture_conditions(row, lecture_id):
    cursor.execute('''
    INSERT OR REPLACE INTO LectureConditions 
    (lectureID, canTakeForeignPeople, canTakeMultipleMajor, canTakeOnly1year, 
    canTakeOnly2year, canTakeOnly3year, canTakeOnly4year, canTakeOnly5year, canTakeBunBan, 
    majorRecogBunBan, canTakeOnlyAthlete, canTakeOnlyChambit, requirementClass)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        lecture_id,
        row['외국인_수강가능'],  # canTakeForeignPeople (기본값 설정 필요)
        1,  # canTakeMultipleMajor (기본값 1)
        row['1학년'],
        row['2학년'],
        row['3학년'],
        row['4학년'],
        0,
        row['수강가능한 학과'],
        row['전공 인정 학과'],
        0,  # canTakeOnlyAthlete (기본값 0)
        0,  # canTakeOnlyChambit (기본값 0)
        row['선수과목']
    ))

# 각 행에 대해 처리
for _, row in df.iterrows():
    # LectureList에서 lectureID 찾기
    cursor.execute('''
    SELECT lectureID FROM LectureList
    WHERE year = ? AND semester = ? AND lecNumber = ?
    ''', ('24', '2학기', row['학정번호']))
    
    lecture_id_row = cursor.fetchone()
    
    if lecture_id_row:
        lecture_id = lecture_id_row[0]
        insert_lecture_conditions(row, lecture_id)

# 변경사항 커밋 및 연결 종료
conn.commit()
conn.close()

print("LectureConditions 테이블 업데이트가 완료되었습니다.")

LectureConditions 테이블 업데이트가 완료되었습니다.


In [2]:
import csv
import sqlite3
import pandas as pd

def print_debug(message):
    print(f"DEBUG: {message}")

try:
    # 데이터베이스 연결
    conn = sqlite3.connect('kwu-lecture-recommendation-v5.db')
    cursor = conn.cursor()
    print_debug("데이터베이스에 연결되었습니다.")

    # CSV 파일 읽기
    df = pd.read_csv('processed_timetable.csv', encoding='cp949')
    df = df.fillna('')
    print_debug(f"CSV 파일을 읽었습니다. 총 {len(df)} 행이 있습니다.")

    def safe_int(value, default=0):
        try:
            return int(value)
        except (ValueError, TypeError):
            return default

    # 데이터베이스에 삽입하는 함수
    def insert_lecture_conditions(row, lecture_id):
        try:
            cursor.execute('''
            INSERT OR REPLACE INTO LectureConditions 
            (lectureID, canTakeForeignPeople, canTakeMultipleMajor, canTakeOnly1year, 
            canTakeOnly2year, canTakeOnly3year, canTakeOnly4year, canTakeOnly5year, canTakeBunBan, 
            majorRecogBunBan, canTakeOnlyAthlete, canTakeOnlyChambit, requirementClass)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                lecture_id,
                safe_int(row.get('외국인_수강가능', 0)),
                1,  # canTakeMultipleMajor (기본값 1)
                safe_int(row.get('1학년', 0)),
                safe_int(row.get('2학년', 0)),
                safe_int(row.get('3학년', 0)),
                safe_int(row.get('4학년', 0)),
                0,  # canTakeOnly5year (기본값 0)
                str(row.get('수강가능한 학과', '')),
                str(row.get('전공 인정 학과', '')),
                0,  # canTakeOnlyAthlete (기본값 0)
                0,  # canTakeOnlyChambit (기본값 0)
                str(row.get('선수과목', ''))
            ))
            return True
        except sqlite3.Error as e:
            print_debug(f"Error inserting data for lecture ID {lecture_id}: {e}")
            print_debug(f"Problematic row: {row}")
            return False

    # LectureList 테이블의 모든 lectureID 가져오기
    cursor.execute("SELECT lectureID, year, semester, lecNumber FROM LectureList")
    lecture_list = {(year, semester, lecNumber): lectureID for lectureID, year, semester, lecNumber in cursor.fetchall()}
    print_debug(f"LectureList에서 {len(lecture_list)}개의 강의 정보를 가져왔습니다.")

    success_count = 0
    error_count = 0

    # 각 행에 대해 처리
    for index, row in df.iterrows():
        year = 24
        semester = '2학기'
        lec_number = str(row['학정번호'])
        
        lecture_id = lecture_list.get((year, semester, lec_number))
        
        if lecture_id:
            if insert_lecture_conditions(row, lecture_id):
                success_count += 1
                if success_count % 100 == 0:  # 100개마다 진행상황 출력
                    print_debug(f"{success_count}개의 강의 정보가 성공적으로 업데이트되었습니다.")
            else:
                error_count += 1
        else:
            print_debug(f"강의를 찾을 수 없습니다: 년도={year}, 학기={semester}, 학정번호={lec_number}")

    print_debug(f"처리 완료: 성공 {success_count}개, 실패 {error_count}개")

    # 변경사항 커밋 및 연결 종료
    conn.commit()
    conn.close()
    print_debug("데이터베이스 연결이 종료되었습니다.")

except Exception as e:
    print_debug(f"예외가 발생했습니다: {e}")

print("LectureConditions 테이블 업데이트 프로세스가 완료되었습니다.")

DEBUG: 데이터베이스에 연결되었습니다.
DEBUG: CSV 파일을 읽었습니다. 총 1296 행이 있습니다.
DEBUG: LectureList에서 2718개의 강의 정보를 가져왔습니다.
DEBUG: 100개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 200개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 300개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 400개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 500개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 600개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 700개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 800개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 900개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 1000개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 1100개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 1200개의 강의 정보가 성공적으로 업데이트되었습니다.
DEBUG: 처리 완료: 성공 1296개, 실패 0개
DEBUG: 데이터베이스 연결이 종료되었습니다.
LectureConditions 테이블 업데이트 프로세스가 완료되었습니다.
